In [1]:
import numpy as np 
import numba 

In [68]:
_N = 8
N = _N**2
lattice = np.zeros(N, dtype=np.int64)
empty_mask = np.zeros(N, dtype=np.bool_)
pred_mask = np.zeros(N, dtype=np.bool_)
prey_mask = np.zeros(N, dtype=np.bool_)
# Fill
Nprey, Npred = int(N // 5), int(N // 5)
indices = np.random.choice(N, size=Nprey+Npred, replace=False)
prey_idxs = indices[:Nprey]
pred_idxs = indices[Nprey:]
prey_mask[prey_idxs] = True 
pred_mask[pred_idxs] = True 

In [75]:
lattice[prey_idxs] = -1 
lattice[pred_idxs] = 1
prey_indices = np.where(prey_mask)[0]
prey_indices

array([ 7,  8, 10, 12, 14, 25, 26, 30, 37, 44, 49, 54])